In [98]:
import requests
import pandas as pd
import numpy as np

In [28]:
# Import list of ICOs
file = 'list_of_features - ICOs_reddit.csv'
df1 = pd.read_csv(file)
df1.head()

,ICO,# Reddit Posts
0,RequestNetwork,986
1,ArkEcosystem,980
2,ETHLend,977
3,RebelliousCoin,976
4,RedPulseToken,951


In [76]:
### Set Up Mongo Connection

import urllib.parse
from pymongo import MongoClient

username = urllib.parse.quote_plus('w210_db_user')
password = urllib.parse.quote_plus('q1w2e3r4$')
conn = MongoClient('mongodb://%s:%s@198.11.212.212:27017/w210_db' % (username, password))
db = conn.w210_db

deadcoin = db["deadcoin_ico"]
result = deadcoin.find({})

data = []
for i,row in enumerate(result):
    data.append(row)
dead_icos = pd.DataFrame(data)

In [99]:
response = requests.get("http://198.11.212.212:8010/_summary_data1")
response.json()

[{'company_url': 'https://patron-ico.io/',
  'description': 'A decentralized platform which shares economies of influencers all over the world',
  'ico_dates': '09/05/2018 00:00:00 UTC - 09/06/2018',
  'ico_sold_status': 'ICO: 240,000,000 PAT of 400,000,000',
  'name': 'PATRON',
  'owner_linkedin_profile': 'https://www.linkedin.com/in/atsushi-hisatsumi/',
  'owner_name': 'Atsushi Hisatsumi',
  'status': 'no_reddit',
  'tracker_url': 'https://icotracker.net/project/patron',
  'whitepaper': 'https://patron-ico.io/doc/patron-wp-en-01-20180423.pdf'},
 {'company_url': 'http://crypton.vc/',
  'description': 'A Blockchain-based cryptocurrency venture capital fund',
  'ico_dates': '29/03/2018 00:00:00 UTC - 09/06/2018',
  'ico_sold_status': 'ICO: 50,400,000 CRN of 72,000,000',
  'name': 'Crypton VC',
  'owner_linkedin_profile': 'https://www.linkedin.com/in/jaceknowak2/',
  'owner_name': 'Jacek Nowak',
  'status': 'no_reddit',
  'tracker_url': 'https://icotracker.net/project/crypton_vc',
  'whi

### CryptoCompare

In [42]:
# GET request to CryptoCompare
response = requests.get("https://www.cryptocompare.com/api/data/coinlist/")
# Result comes back as a flat dict, transform to a list of dicts
data = []
for k,v in response.json()['Data'].items():
    data.append(v)
# Visualize as a dataframe 
df2 = pd.DataFrame(data)
df2.head()

,Algorithm,CoinName,FullName,FullyPremined,Id,ImageUrl,Name,PreMinedValue,ProofType,SortOrder,Sponsored,Symbol,TotalCoinSupply,TotalCoinsFreeFloat,Url
0,Scrypt,Dogecoin,Dogecoin (DOGE),0,4432,/media/19684/doge.png,DOGE,N/A,PoW,8,False,DOGE,N/A,N/A,/coins/doge/overview
1,Equihash,ZCash,ZCash (ZEC),0,24854,/media/351360/zec.png,ZEC,N/A,PoW,9,False,ZEC,21000000,N/A,/coins/zec/overview
2,SHA-512,Bitshares,Bitshares (BTS),1,5039,/media/20705/bts.png,BTS,N/A,PoS,10,False,BTS,2511953117,N/A,/coins/bts/overview
3,Multiple,DigiByte,DigiByte (DGB),0,4430,/media/12318264/7638-nty_400x400.jpg,DGB,N/A,PoW,11,False,DGB,21000000000,N/A,/coins/dgb/overview
4,SHA256,BitcoinDark,BitcoinDark (BTCD),0,4400,/media/19630/btcd_1.png,BTCD,N/A,PoW/PoS,13,False,BTCD,22000000,N/A,/coins/btcd/overview


### CryptoWatch

In [51]:
# GET request to CryptoCompare
response = requests.get("https://api.cryptowat.ch/assets")
data = response.json()['result']
df3 = pd.DataFrame(data)
df3.head()

,fiat,id,name,route,symbol
0,False,404,FirstBlood,https://api.cryptowat.ch/assets/1st,1st
1,False,201,ICON,https://api.cryptowat.ch/assets/ICX,ICX
2,False,517,The Abyss,https://api.cryptowat.ch/assets/abyss,abyss
3,False,502,Alphacat,https://api.cryptowat.ch/assets/acat,acat
4,False,484,Achain,https://api.cryptowat.ch/assets/act,act


In [96]:
# Lowercase names
datasets = [
    {'name': 'DeadICO','data': dead_icos, 'column': 'name'},
    {'name': 'reddit', 'data': df1, 'column': 'ICO'},
    {'name': 'CryptoCompare', 'data': df2, 'column': 'CoinName'},
    {'name': 'CryptoWatch','data': df3, 'column': 'name'}
]
df = datasets[0]
for i,d in enumerate(datasets):
    d['data']['coin_name'] = d['data'][d['column']].apply(lambda x: x.lower())
    if i == 0:
        df = d['data']
        df[d['name']] = 1
    else:
        ind = 'ind_%d' % i
        df = pd.merge(df,d['data'],on='coin_name',how='outer',indicator=ind)
        df[d['name']] = df[ind].apply(lambda x: 0 if x == 'left_only' else 1)
        df.drop(columns=[ind])

In [97]:
df.groupby([x['name'] for x in datasets]).aggregate('count')[['coin_name']].reset_index()

,DeadICO,reddit,CryptoCompare,CryptoWatch,coin_name
0,1.0,0.0,0.0,0,618
1,1.0,0.0,0.0,1,3
2,1.0,0.0,1.0,0,190
3,1.0,0.0,1.0,1,6
4,1.0,1.0,0.0,0,2


In [93]:
compare = pd.merge(dead_icos,df2,on='coin_name',how='outer',indicator=True)
compare.groupby(['_merge']).aggregate('count')

,_id,comment,name,rating,status,url,coin_name,Algorithm,CoinName,FullName,...,ImageUrl,Name,PreMinedValue,ProofType,SortOrder,Sponsored,Symbol,TotalCoinSupply,TotalCoinsFreeFloat,Url
_merge,,,,,,,,,,,,,,,,,,,,,
left_only,623,623,623,623,623,623,623,0,0,0,...,0,0,0,0,0,0,0,0,0,0
right_only,0,0,0,0,0,0,2496,2496,2496,2496,...,2493,2496,2496,2496,2496,2496,2496,2496,2496,2496
both,196,196,196,196,196,196,196,196,196,196,...,196,196,196,196,196,196,196,196,196,196


In [79]:
response = requests.get("https://api.coinmarketcap.com/v2/ticker/1/")
response.json()


{'data': {'id': 1,
  'name': 'Bitcoin',
  'symbol': 'BTC',
  'website_slug': 'bitcoin',
  'rank': 1,
  'circulating_supply': 17105725.0,
  'total_supply': 17105725.0,
  'max_supply': 21000000.0,
  'quotes': {'USD': {'price': 6796.47,
    'volume_24h': 3703140000.0,
    'market_cap': 116258546791.0,
    'percent_change_1h': 0.28,
    'percent_change_24h': 2.45,
    'percent_change_7d': 5.62}},
  'last_updated': 1529549076},
 'metadata': {'timestamp': 1529548813, 'error': None}}

In [80]:
dead_icos

,_id,comment,name,rating,status,url,coin_name
0,5b22206e48d621122074eb0f,,Swagcoin,0.0 rating,Parody,,swagcoin
1,5b22206e48d621122074eb10,Another coin name chosen specifically to elimi...,StarvingArtistCoin,0.0 rating,Parody,,starvingartistcoin
2,5b22206e48d621122074eb11,Observed trading with market cap $23 on 2 June...,Sha1coin,0.0 rating,Parody,,sha1coin
3,5b22206e48d621122074eb12,"Um. So what was this, a play to appeal to pro...",Obama_bin_lotteryCoin,0.0 rating,Parody,,obama_bin_lotterycoin
4,5b22206e48d621122074eb13,,Mariocoin,0.0 rating,Parody,,mariocoin
5,5b22206e48d621122074eb14,,Koindashian,0.0 rating,Parody,,koindashian
6,5b22206e48d621122074eb15,and somebody else thought misspelling it was a...,JezusCoin,0.0 rating,Parody,,jezuscoin
7,5b22206f48d621122074eb16,created via Coingen.,HackerNewsCoin,0.0 rating,Parody,,hackernewscoin
8,5b22206f48d621122074eb17,,FourtyTwoCoin,0.0 rating,Parody,,fourtytwocoin
9,5b22206f48d621122074eb18,,Dollarpounds,0.0 rating,Parody,,dollarpounds
